In [1]:
### ------------------------------------------------------------------------
### Trabalho Final  Inteligência Artificial
### Adriano G e Jean M.
### -------------------------------------------------------------------------

In [2]:
import h1 as h1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics  as sts
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

ModuleNotFoundError: No module named 'h1'

In [ ]:
df=pd.read_csv('Titanic-Dataset.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

## Resumo geral da base de dados

In [ ]:
print ("Linhas: " , df.shape[0])
print ("Colunas: " , df.shape[1])
print ("\nAtributos : \n" , df.columns.tolist())
print ("\nValores faltantes :  ", df.isnull().sum().values.sum())
print ("\nValores únicos :  \n",df.nunique())

## Pré-processamento

### Remoção ID feature e verificação de dados faltantes

In [ ]:
# Verifica a quantidade de dados faltrantes
df.isna().sum()


In [ ]:
### Substituição NAs Age pela mediana
mediana = df['Age'].median()
#preenche NAs
df['Age'].fillna(mediana, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df['Embarked'].fillna('S', inplace=True)

### Carregar dados para previsao da Cabine com um treino de random forest

In [ ]:
### crio uma nova variavel com as colunas relevantes para previsao
df_cabines = df[['Pclass','Sex','Age','Fare','Cabin','Embarked']]
df_cabines


In [ ]:
### separo em cabines conhecidas e cabines desconhecidas
df_conhecidas = df_cabines.dropna(subset=['Cabin'])
df_desconhecidas = df_cabines[df_cabines['Cabin'].isna()]
df_desconhecidas


In [ ]:
df_conhecidas

In [ ]:
### removendo a coluna cabin para separarmos atributos e rotulos
### Separar os dados conhecidos em atributos e rótulos
atributos_conhecidos = df_conhecidas.drop('Cabin', axis=1)
cabines_conhecidas = df_conhecidas['Cabin']


In [ ]:
### convertendo para Dummy
atributos_conhecidos = pd.get_dummies(atributos_conhecidos)
cabines_conhecidas = pd.get_dummies(cabines_conhecidas)


In [ ]:
###treinar com o modelo random forest
modelo = RandomForestClassifier()
modelo.fit(atributos_conhecidos,cabines_conhecidas)

In [ ]:
###prever valores faltantes na coluna cabin
atributos_desconhecidos = df_desconhecidas.drop('Cabin',axis=1)
atributos_desconhecidos = pd.get_dummies(atributos_desconhecidos)
prever_cabine = modelo.predict(atributos_desconhecidos)
#prever_cabine = prever_cabine[:, 0]


prever_cabine

In [ ]:
# Criar DataFrame com as previsões codificadas
df_previsoes = pd.DataFrame(prever_cabine, columns=[f'Cabin_{i}' for i in range(len(modelo.classes_))])

df_previsoes_revertidas = df_previsoes.idxmax(axis=1)

df.loc[df['Cabin'].isna(), 'Cabin'] = df_previsoes_revertidas.values





In [ ]:
# Verifica a quantidade de dados faltrantes
df.isna().sum()

In [ ]:
df

In [ ]:
df=df.drop(columns='PassengerId',axis=1)
df=df.drop(columns='Ticket',axis=1)
df=df.drop(columns='Name',axis=1)
df

Atributos com mais de 2 valores,


In [ ]:
df = pd.get_dummies(df)



In [ ]:
df.columns

In [ ]:
std=StandardScaler()
columns = ['Age','Fare']
scaled = std.fit_transform(df[['Age','Fare']])
scaled = pd.DataFrame(scaled,columns=columns)
df=df.drop(columns=columns,axis=1)

In [ ]:
df=df.merge(scaled, left_index=True, right_index=True, how = "right")
df

In [ ]:
X = df.drop(['Survived'], axis=1).values
y = df['Survived'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
#Aplica balanceamento nas classes
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE()
x_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

In [ ]:
print(x_train_oversampled.shape)
print(X_train.shape)

In [ ]:
### KNN Classifier

In [ ]:
df.columns

In [ ]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k,metric = 'euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

In [ ]:
y_proba = knn.predict_proba(X_test)

In [ ]:
y_proba

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
# Teste para diferentes valores de k
k_range = list(range(1,21))
scores = []
for k in k_range:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
    knn = KNeighborsClassifier(n_neighbors=k,metric = 'euclidean')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))

plt.plot(k_range, scores)
plt.xlabel('Value of k for KNN')
plt.ylabel('Accuracy Score')
plt.title('Accuracy Scores for Values of k of k-Nearest-Neighbors')
plt.show()

In [ ]:
## Normalização dos dados

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
# experimenting with different n values
k_range = list(range(1,21))
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))

plt.plot(k_range, scores)
plt.xlabel('Value of k for KNN')
plt.ylabel('Accuracy Score')
plt.title('Accuracy Scores for Values of k of k-Nearest-Neighbors')
plt.show()

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Random Forest Classifier</h1>

In [ ]:
forest = RandomForestClassifier(random_state=42)
#==
forest.fit(x_train_oversampled, y_train_oversampled)
#==
#Score
#==
forest_score = forest.score(x_train_oversampled, y_train_oversampled)
forest_test = forest.score(X_test, y_test)
#==
#testing model
#==
y_pred = forest.predict(X_test)
#==
#evaluation
#==
cm = confusion_matrix(y_test,y_pred)
print('Training Score',forest_score)
print('Testing Score \n',forest_test)
print(cm)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

###################################################################################################################################################

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
# param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
#               'max_features':['sqrt','log2'],
#               'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 300]}
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
              'max_features':['sqrt','log2'],
              'n_estimators': [10, 20, 30, 60]}
#### Exemplo com GridSearchCV
# - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
g_search = GridSearchCV(estimator=forest, param_grid=param_grid,
                        cv=10, return_train_score=True)
sm = SMOTE()
x_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
print(np.unique(y_train_oversampled, return_counts=True))
g_search.fit(x_train_oversampled, y_train_oversampled)
print(g_search.best_params_)
print(g_search.best_score_)

#### Exemplo com GridSearchCV
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
g_search = GridSearchCV(estimator = forest, param_grid = param_grid,
                        cv = 10, return_train_score=True)

In [ ]:
sm = SMOTE()
x_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

In [ ]:
print(np.unique(y_train_oversampled, return_counts=True))

In [ ]:
g_search.fit(x_train_oversampled, y_train_oversampled);
print(g_search.best_params_)

In [ ]:
print(g_search.best_score_)

#### Exemplo com RandomizedSearchCV
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import KMeans

In [ ]:
r_search = RandomizedSearchCV(estimator = forest, param_distributions = param_grid,
                        n_iter= 10, cv = 10, return_train_score=True)

In [ ]:
sm = SMOTE()
x_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

In [ ]:
print(np.unique(y_train_oversampled, return_counts=True))

In [ ]:
r_search.fit(x_train_oversampled, y_train_oversampled);
print(r_search.best_params_)

In [ ]:
print(r_search.best_score_)